In [1]:
from hamiltonian import HamiltonianSmall, Hamiltonian

import var_opt_lagrange
import time

# Lagrange by Rudy

See Rudy's note on optimizing the variance with Lagrange multiplier method

This notebook works for both convex cost function `diagonal` and non-convex cost function `mixed`

For the non-convex setting, we need access to the Hartree-Fock bitstrings. Currently, we only have that for JW encodings.

In [2]:
lih_small = HamiltonianSmall('LiH', 1.5) # this encoding uses reduction techniques
beh2_small = HamiltonianSmall('BeH2', 1.3) # this encoding uses reduction techniques

# 4 qubits
h2_jw_4 = Hamiltonian('H2_STO3g_4qubits', 'jw')
h2_parity_4 = Hamiltonian('H2_STO3g_4qubits', 'parity')
h2_bk_4 = Hamiltonian('H2_STO3g_4qubits', 'bk')

# 8 qubits
h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

# 12 qubits
lih_jw = Hamiltonian('LiH_STO3g_12qubits', 'jw')

# 14 qubits
h2o_jw = Hamiltonian('H2O_STO3g_14qubits', 'jw')

beh2_jw = Hamiltonian('BeH2_STO3g_14qubits', 'jw')

# 16 qubits
nh3_jw = Hamiltonian('NH3_STO3g_16qubits', 'jw')

# 20 qubits
c2_jw = Hamiltonian('C2_STO3g_20qubits', 'jw')

In [3]:
hamiltonians = {"lih_small": lih_small,
                "beh2_small": beh2_small,
                "h2_jw_4": h2_jw_4,
                "h2_parity_4": h2_parity_4,
                "h2_bk_4": h2_bk_4,
                "h2_jw": h2_jw,
                "h2_parity": h2_parity,
                "h2_bk": h2_bk,
                "lih_jw": lih_jw,
                "h2o_jw": h2o_jw,
                "beh2_jw": beh2_jw,
                "nh3_jw": nh3_jw,
                "c2_jw": c2_jw}

In [4]:
method = 'lagrange'

An example showing how to call this from `Hamiltonian.PauliRep.local_dists_optimal`

In [5]:
from numpy.random import rand
import numpy as np

np.random.seed(10)

def isProbability(betas, tol=1.0e-5):
    """
        Check if betas are probabilities
    """
    for k,v in betas.items():
        for _v in v:
            if _v < 0:
                return False
        if np.abs(np.sum(v) - 1.0) > tol:
            print("\tErr:", np.abs(np.sum(v) - 1.0) )
            return False
    return True

def get_rand_betas(num_qubits):
    """
        generate random initial points to search for betas
    """
    betas = {}
    for qubit in range(num_qubits):
        betas[qubit] = rand(3)
        betas[qubit] = betas[qubit]/np.sum(betas[qubit])
    return betas

In [6]:
def both_variances_lagrange(ham, num_trials=3, display_betas=False):
    t0 = time.time()
    energy, state = ham.pauli_rep.ground(multithread=True) # you can add optional keyword num_cores=15
    t1 = time.time()
    # diagonal objective function
    
    objective = 'diagonal'
    β_optimal_diagonal = ham.pauli_rep.local_dists_optimal(objective, method)
    if display_betas:
        print("\tbeta (diagonal):", β_optimal_diagonal)
    t2 = time.time()
    var = ham.pauli_rep.variance_local(energy, state, β_optimal_diagonal)
    t3 = time.time()
    print("Diagonal variance:", var)
    print("\n")
    
    # mixed objective function
    
    objective = 'mixed'
    assert ham.encoding == 'jw' # we do not have HF bitstrings for other encoding at the moment.
    bitstring_HF = ham.read_bitstring_HF()
    
    for trial in range(num_trials): 
        #because we do not have convexity, we perform several trials to look for optimal betas
        print("Trial:", _)
        if trial == 0:
            beta_init = β_optimal_diagonal
        else:
            beta_init = get_rand_betas(ham.pauli_rep.num_qubits)
        
        if trial == num_trials-1:
            t4 = time.time()
        
        β_optimal_mixed = ham.pauli_rep.local_dists_optimal(objective, method, 
                                                            bitstring_HF=bitstring_HF, β_initial=beta_init)
        
        if not isProbability(β_optimal_mixed):
            print("WARNING: not probability:", β_optimal_mixed)
        if display_betas:
            print("\tbeta (mixed):", β_optimal_diagonal)        
        
        if trial == num_trials-1:
            t5 = time.time()
            
        var = ham.pauli_rep.variance_local(energy, state, β_optimal_mixed)
        
        if trial == num_trials-1:
            t6 = time.time()
        
        print("Mixed variance:", var)
        
        print("\n")
        
    print("Timings:")
    print("ground state: {} seconds".format(int(t1-t0)))
    print("optimal diagonal beta: {} seconds".format(int(t2-t1)))
    print("calculate variance: {} seconds".format(int(t3-t2)))
    print("optimal mixed beta (last trial): {} seconds".format(int(t5-t4)))
    print("calculate variance (last trial): {} seconds".format(int(t6-t5)))    

In [7]:
both_variances_lagrange(h2_jw_4)

Diagonal variance: 1.8555830498849535


Trial: 
Mixed variance: 1.8546562200043255


Trial: 
Mixed variance: 1.854656258586077


Trial: 
Mixed variance: 1.854656259066888


Timings:
ground state: 0 seconds
optimal diagonal beta: 0 seconds
calculate variance: 0 seconds
optimal mixed beta (last trial): 0 seconds
calculate variance (last trial): 0 seconds


In [8]:
both_variances_lagrange(h2_jw)

Diagonal variance: 17.74194781113713


Trial: 
Mixed variance: 17.457187282027824


Trial: 
Mixed variance: 17.45719122547085


Trial: 
Mixed variance: 17.457191457394067


Timings:
ground state: 0 seconds
optimal diagonal beta: 0 seconds
calculate variance: 0 seconds
optimal mixed beta (last trial): 0 seconds
calculate variance (last trial): 0 seconds


In [9]:
both_variances_lagrange(lih_jw)

Diagonal variance: 14.792751908499572


Trial: 
Mixed variance: 14.877408442706624


Trial: 
Mixed variance: 14.87755639740817


Trial: 
Mixed variance: 14.877562484247587


Timings:
ground state: 0 seconds
optimal diagonal beta: 1 seconds
calculate variance: 20 seconds
optimal mixed beta (last trial): 9 seconds
calculate variance (last trial): 22 seconds


In [10]:
#both_variances_lagrange(h2o_jw)